# Lesson 1: Router Engine

Welcome to Lesson 1.

To access the `requirements.txt` file, the data/pdf file required for this lesson and the `helper` and `utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

第一步，加载openai的key，通过预设参数的方式

In [1]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

如果是自己用过的话，直接用下面的方式进行赋值

In [1]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "   "
openai.api_key = os.environ["OPENAI_API_KEY"]


In [ ]:
异步处理，更快速

In [2]:
import nest_asyncio

nest_asyncio.apply()

## Load Data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

In [3]:

from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

## Define LLM and Embedding model

In [ ]:
经典的Embedding模型，没有什么花活

In [4]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [5]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Define Summary Index and Vector Index over the Same Data

In [6]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Define Query Engines and Set Metadata

In [7]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

装了两个Query_engine

In [8]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)



In [9]:
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

## Define Router Query Engine

目前的感受是，在查询前，会判断查询的条件是总结式，还是搜索式，通过对问题的判断，自主选择一个工具去执行任务。体现了Agent的前置判断能力以及智能化。

In [10]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [11]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Useful for summarization questions related to MetaGPT.
The document introduces MetaGPT, a meta-programming framework for multi-agent collaboration based on Large Language Models (LLMs). It emphasizes role specialization, structured communication interfaces, and executable feedback mechanisms to enhance code generation quality during runtime. MetaGPT models a group of agents as a simulated software company, incorporating Standardized Operating Procedures (SOPs) to streamline workflows and improve collaboration efficiency. The framework achieves state-of-the-art performance in evaluations, outperforming previous approaches, and offers a promising approach to developing LLM-based multi-agent systems. Additionally, it explores recursive self-improvement mechanisms, an "Economy of Minds" (EOM) for assigning credit in agent societies, and the development process of software applications like the "Drawing App" using MetaGPT. The document also discusses the performanc

In [12]:
print(len(response.source_nodes))

34


In [13]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 0: MetaGPT is a model designed for summarization tasks, which may involve sharing information between agents..
Agents share information with other agents by utilizing a shared message pool to publish structured messages. They can also subscribe to relevant messages based on their profiles, allowing for efficient communication and collaboration within the multi-agent system. Additionally, agents share information through mechanisms such as message pools and subscriptions within the MetaGPT framework, enabling efficient sharing of information among agents to enhance collaboration and problem-solving capabilities within the simulated software company environment.


## Let's put everything together

In [ ]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("metagpt.pdf")

In [14]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))

Selecting query engine 1: The ablation study results are specific context from the MetaGPT paper, making choice 2 the most relevant..
The ablation study results provide insights into the impact of removing certain components or features from a system or model. This analysis helps in understanding the contribution and importance of individual elements towards the overall performance or functionality of the system.


In [15]:
response = query_engine.query("how many roles in Metagpt?")
print(str(response))

Selecting query engine 1: The question is asking for a specific detail about MetaGPT, which is the number of roles. Choice 2 is more relevant for retrieving specific context from the MetaGPT paper..
MetaGPT defines five roles: Product Manager, Architect, Project Manager, Engineer, and QA Engineer.
